In [57]:
#importing the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pystan
import scipy.stats as sts
sns.set()

In [58]:

#importing the data and previewing
data = pd.read_csv('/Users/anirudhn/Desktop/Code/CS146_Assignments/LBA/Data.csv', na_values='n/a')
data.head(n=3)

,Timestamp,Email Address,Your name,Country where you (and the grocery store) are,"Currency (e.g. EUR, GBP, USD)",Grocery store name and street address (or URL if shopping online),Price perception of the store brand,Average rental price near the grocery store,Product 1 quantity (kg),Product 1 price,...,Product 2 quantity (count),Product 2 price.8,Product 3 quantity (count),Product 3 price.8,Product 1 quantity (kg).7,Product 1 price.9,Product 2 quantity (kg).7,Product 2 price.9,Product 3 quantity (kg).7,Product 3 price.9
0,10/20/2020 23:22:49,tiagoflora@minerva.kgi.edu,Tiago Flora,Germany,EUR,"EDEKA, Grunerstraße 20",Mid-range,EUR900,1.0,2.93,...,6.0,1.75,4.0,1.99,0.312,3.97,0.60,3.87,0.400,2.90
1,10/20/2020 23:32:18,tiagoflora@minerva.kgi.edu,Tiago Flora,Germany,EUR,"EDEKA Andreasstraße 59, 10243 Berlin",Mid-range,EUR850,1.0,1.95,...,6.0,2.73,10.0,1.63,0.394,5.02,0.43,7.17,0.311,9.12
2,10/23/2020 18:39:04,katja.dellalibera@minerva.kgi.edu,Katja Della Libera,Germany,EUR,"EDEKA BAUR, Gottlieber Str. 34, 78462 Konstanz",Mid-range,1000,1.0,2.93,...,10.0,3.23,10.0,1.25,0.600,3.88,0.40,2.91,NaN,NaN


In [59]:
#dropping the columns that are not required for the initial part of the assignment
data.drop(['Timestamp','Email Address','Your name','Grocery store name and street address (or URL if shopping online)','Average rental price near the grocery store'], axis = 1, inplace = True)

#naming the columns in a more convenient manner
data.rename(columns={"Country where you (and the grocery store) are": "Country", "Currency (e.g. EUR, GBP, USD)": "Currency","Price perception of the store brand":"Brand Perception"}, inplace= True)

#previewing the data
data.head()

,Country,Currency,Brand Perception,Product 1 quantity (kg),Product 1 price,Product 2 quantity (kg),Product 2 price,Product 3 quantity (kg),Product 3 price,Product 1 quantity (kg).1,...,Product 2 quantity (count),Product 2 price.8,Product 3 quantity (count),Product 3 price.8,Product 1 quantity (kg).7,Product 1 price.9,Product 2 quantity (kg).7,Product 2 price.9,Product 3 quantity (kg).7,Product 3 price.9
0,Germany,EUR,Mid-range,1.0,2.93,2.0,3.22,1.0,1.75,1.0,...,6.0,1.75,4.0,1.99,0.312,3.97,0.600,3.87,0.400,2.90
1,Germany,EUR,Mid-range,1.0,1.95,1.0,2.93,0.8,2.93,1.0,...,6.0,2.73,10.0,1.63,0.394,5.02,0.430,7.17,0.311,9.12
2,Germany,EUR,Mid-range,1.0,2.93,1.0,1.25,2.0,3.42,1.0,...,10.0,3.23,10.0,1.25,0.600,3.88,0.400,2.91,NaN,NaN
3,Germany,EUR,Luxury (expensive),1.0,3.99,1.0,3.99,1.0,3.99,1.0,...,10.0,1.99,1.0,0.45,1.000,32.90,NaN,NaN,NaN,NaN
4,Germany,EUR,Budget (cheap),1.0,2.93,1.0,2.93,1.0,2.93,1.0,...,6.0,2.44,10.0,2.34,0.600,3.87,0.392,4.99,0.333,3.26


In [60]:
for i in range(4,62,2):
    print(data.iloc[0,i])

2.93
3.22
1.75
1.73
1.73
nan
1.95
2.24
2.84
0.96
1.05
1.93
1.75
1.26
1.56
1.95
2.83
4.89
1.46
1.05
1.25
2.15
2.14
2.1
3.79
1.75
1.99
3.97
3.87


In [61]:
#converting all prices to a unified currency:USD
for index in data.index:
    if data.iloc[index,1] == "EUR":
        for i in range(4,62,2):
            data.iloc[index,i] = data.iloc[index,i]*1.187
    elif data.iloc[index,1] == "GBP":
        for k in range(4,62,2):
            data.iloc[index,k] = data.iloc[index,k]*1.316
    elif data.iloc[index,1] == "MAD":
        for m in range(4,62,2):
            data.iloc[index,m] = data.iloc[index,m]*0.109
    elif data.iloc[index,1] == "VND":
        for l in range(4,62,2):
            data.iloc[index,l] = data.iloc[index,l]*0.000043
    elif data.iloc[index,1] == "KRW":
        for h in range(4,62,2):
            data.iloc[index,h] = data.iloc[index,h]*0.00089
    elif data.iloc[index,1] == "GTQ":
        for g in range(4,62,2):
            data.iloc[index,g] = data.iloc[index,g]*0.129

data.head(5)

,Country,Currency,Brand Perception,Product 1 quantity (kg),Product 1 price,Product 2 quantity (kg),Product 2 price,Product 3 quantity (kg),Product 3 price,Product 1 quantity (kg).1,...,Product 2 quantity (count),Product 2 price.8,Product 3 quantity (count),Product 3 price.8,Product 1 quantity (kg).7,Product 1 price.9,Product 2 quantity (kg).7,Product 2 price.9,Product 3 quantity (kg).7,Product 3 price.9
0,Germany,EUR,Mid-range,1.0,3.47791,2.0,3.82214,1.0,2.07725,1.0,...,6.0,2.07725,4.0,2.36213,0.312,4.71239,0.600,4.59369,0.400,2.90
1,Germany,EUR,Mid-range,1.0,2.31465,1.0,3.47791,0.8,3.47791,1.0,...,6.0,3.24051,10.0,1.93481,0.394,5.95874,0.430,8.51079,0.311,9.12
2,Germany,EUR,Mid-range,1.0,3.47791,1.0,1.48375,2.0,4.05954,1.0,...,10.0,3.83401,10.0,1.48375,0.600,4.60556,0.400,3.45417,NaN,NaN
3,Germany,EUR,Luxury (expensive),1.0,4.73613,1.0,4.73613,1.0,4.73613,1.0,...,10.0,2.36213,1.0,0.53415,1.000,39.05230,NaN,NaN,NaN,NaN
4,Germany,EUR,Budget (cheap),1.0,3.47791,1.0,3.47791,1.0,3.47791,1.0,...,6.0,2.89628,10.0,2.77758,0.600,4.59369,0.392,5.92313,0.333,3.26


In [7]:
data.head()

,Country,Currency,Brand Perception,Product 1 quantity (kg),Product 1 price,Product 2 quantity (kg),Product 2 price,Product 3 quantity (kg),Product 3 price,Product 1 quantity (kg).1,...,Product 2 quantity (count),Product 2 price.8,Product 3 quantity (count),Product 3 price.8,Product 1 quantity (kg).7,Product 1 price.9,Product 2 quantity (kg).7,Product 2 price.9,Product 3 quantity (kg).7,Product 3 price.9
0,Germany,EUR,Mid-range,1.0,2.93,2.0,3.22,1.0,1.75,1.0,...,6.0,1.75,4.0,1.99,0.312,3.97,0.600,3.87,0.400,2.90
1,Germany,EUR,Mid-range,1.0,1.95,1.0,2.93,0.8,2.93,1.0,...,6.0,2.73,10.0,1.63,0.394,5.02,0.430,7.17,0.311,9.12
2,Germany,EUR,Mid-range,1.0,2.93,1.0,1.25,2.0,3.42,1.0,...,10.0,3.23,10.0,1.25,0.600,3.88,0.400,2.91,NaN,NaN
3,Germany,EUR,Luxury (expensive),1.0,3.99,1.0,3.99,1.0,3.99,1.0,...,10.0,1.99,1.0,0.45,1.000,32.90,NaN,NaN,NaN,NaN
4,Germany,EUR,Budget (cheap),1.0,2.93,1.0,2.93,1.0,2.93,1.0,...,6.0,2.44,10.0,2.34,0.600,3.87,0.392,4.99,0.333,3.26
